### Prescriptive Models - Control Variables

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

### 1. Hospital admission and quality of service

In [2]:
health_data = pd.read_csv('health_data.csv')
health_data.head()

,patient_id,hospital_id,admin_year,patient_died_dummy,startage,female_dummy
0,1,D,2003,0,81,0
1,2,H,2003,1,67,0
2,3,A,2003,0,54,0
3,4,E,2003,0,81,0
4,5,H,2003,0,69,0


In [3]:
health_data.hospital_id.unique()

array(['D', 'H', 'A', 'E', 'G', 'J', 'B', 'F', 'C', 'I'], dtype=object)

### Question 1

In [4]:
reg1 = smf.ols(formula = 'patient_died_dummy ~ hospital_id', data = health_data)
result = reg1.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     119.3
Date:                Wed, 22 Feb 2023   Prob (F-statistic):          1.75e-220
Time:                        14:41:05   Log-Likelihood:                -7416.5
No. Observations:               24480   AIC:                         1.485e+04
Df Residuals:                   24470   BIC:                         1.493e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0970      0.006  

#### (a) The intercept is the mortality rate (probability of death) at hospital A, which is `0.0970`. The coefficient for hospital D, `0.1882` indicates the additional probability of death compared to the constant term. Mortality rate of hospital D is 18.82 percentage points higher than that of hospital A. The two coefficients are statistically significant too, looking at the p-value.

#### (b) The difference between mortality rates of hospital D and E is obtained by calculating the difference between the coefficients.

In [5]:
result.params[3] - result.params[4]

0.24139049162001222

### Question 2

In [6]:
health2 = health_data[(health_data.hospital_id == 'A') + (health_data.hospital_id == 'B')]
health2.head()

,patient_id,hospital_id,admin_year,patient_died_dummy,startage,female_dummy
2,3,A,2003,0,54,0
6,7,A,2003,0,59,0
12,13,B,2003,0,65,0
15,16,A,2003,0,61,0
19,20,A,2003,0,71,0


In [7]:
regAB = smf.ols(formula = 'patient_died_dummy ~ hospital_id', data = health2)
resultAB = regAB.fit()
print(resultAB.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9377
Date:                Wed, 22 Feb 2023   Prob (F-statistic):              0.333
Time:                        14:41:05   Log-Likelihood:                -1446.8
No. Observations:                6611   AIC:                             2898.
Df Residuals:                    6609   BIC:                             2911.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0970      0.005  

#### (a) The coefficient for hospital B is statistically insignificant due to the p-value, so we cannot make the assumption of a causal effect. Additionally, there are other variables that influence the mortality rates. Specific patients that visit hospital A may have risk factors or conditions different to those that visit hospital B. There may be other factors (for example demographic data of the patients) that are causing the regression to suffer from omitted variable bias. A controlled experiment must be conducted with sufficient randomization, in order to infer a causal effect. Accounting for the differences inpatients and randomizing will eliminate other effects and ensure an accurate estimate of mortality rates between visiting 2 different hospitals.

In [8]:
reg_fem  = smf.ols(formula = 'patient_died_dummy ~ female_dummy', data = health2)
resultfem = reg_fem.fit()
print(resultfem.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     434.0
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           2.08e-93
Time:                        14:41:05   Log-Likelihood:                -1237.0
No. Observations:                6611   AIC:                             2478.
Df Residuals:                    6609   BIC:                             2492.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0615      0.004     15.152   

#### (b) We are overestimating the difference in mortality between the 2 hospitals since the female_dummy is a factor that is influencing the mortality rates too.

From the initial regression,
$$
patient\_died = \beta_0 +\beta_1hospital\_id + e
$$

Due to OVB, the error term can be expressed as:
$$
e = \beta_2female\_dummy + u
$$

$$
E(\hat{\beta}_1) = \beta_1 + \frac{Cov(hospital\_id,e)}{Var(hospital\_id)} =\beta_1+\beta_2\frac{Cov(hospital\_id,female\_dummy)}{Var(hospital\_id)}.
$$

The bias is positive, $Cov(hospital\_id,female\_dummy)$ and $\beta_2$ are positive, we are overestimating the effect of difference in the hospital on mortality.

In [9]:
ctrl  = smf.ols(formula = 'patient_died_dummy ~ hospital_id + female_dummy + startage', data = health2)
ctrld = ctrl.fit()
print(ctrld.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     147.6
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           1.43e-92
Time:                        14:41:05   Log-Likelihood:                -1232.8
No. Observations:                6611   AIC:                             2474.
Df Residuals:                    6607   BIC:                             2501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.1165      0.027  

#### (c) I have included the `female_dummy` and `startage` in the regression to try and get closer to the causal estimate. The hospital B coefficient has changed from 0.0072 in the regression without controls to 0.0114 in the regression with controls. This shows that other variables such as the female_dummy have an effect on the mortalities at different hospitals.\
#### Including the female_dummy and age account for omitted variable bias cause by not including them in the regressions. Female_dummy has a clear effect which we observe through the coefficient, which is also significant given the p-value. Startage doesn't have a great effect.

### 2. Demand estimation

In [10]:
demand_data = pd.read_csv('demand_data.csv')

In [11]:
demand_data.head()

,vendor_id,week,summer_dummy,price,sales
0,1,1,0,2.0,8788.7383
1,1,2,0,3.0,8937.9863
2,1,3,0,3.0,8740.1777
3,1,4,0,3.0,8757.1338
4,1,5,0,3.0,8739.6104


### Question 1

In [12]:
demand_v1 = demand_data[demand_data.vendor_id == 1]

In [13]:
reg2 = smf.ols(formula = 'sales ~ price', data = demand_v1)
result2 = reg2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.3250
Date:                Wed, 22 Feb 2023   Prob (F-statistic):              0.571
Time:                        14:41:05   Log-Likelihood:                -360.33
No. Observations:                  52   AIC:                             724.7
Df Residuals:                      50   BIC:                             728.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8983.8227    145.437     61.771      0.0

In [14]:
reg3 = smf.ols(formula = 'sales ~ price + summer_dummy', data = demand_v1)
result3 = reg3.fit()
print(result3.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     11.42
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           8.49e-05
Time:                        14:41:05   Log-Likelihood:                -350.56
No. Observations:                  52   AIC:                             707.1
Df Residuals:                      49   BIC:                             713.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     9177.5500    128.432     71.458   

We notice the price coefficient becomes more negative (from -31 to -141) when we add the summer dummy to the regression. This implies `omitted variable bias` exists if we don't include `summer_dummy` in the regression.

For the first regression (reg2),
$$
sales = \beta_0 +\beta_1price + e,
$$

The effect of summer_dummy is present in `e` and can be represented as:
$$
e = \beta_2summer\_dummy + u.
$$

So the omitted variable bias formula:
$$
E(\hat{\beta}_1) = \beta_1 + \frac{Cov(price,e)}{Var(price)} =\beta_1+\beta_2\frac{Cov(price,summer\_dummy)}{Var(price)}
$$

#### We overestimate the effect of price on sales when the summer_dummy is not used. Whether it is summer or not will have an effect on ice cream sales, $E(\hat{\beta}_1)$ is positive, $Cov(price,summer\_dummy)$ and $\beta_2$ are positive too, $\beta_1$ is negative.
#### When we include the summer dummy, OVB due to summer_dummy is no longer present and we get a more accurate estimate of the price coefficient which is not as accurate in the first regression. Including summer_dummy captures the seasonal variation in sales that is correlated with price.

### Question 2

In [15]:
demand_v2 = demand_data[demand_data.vendor_id == 2]

In [16]:
reg4 = smf.ols(formula = 'sales ~ price', data = demand_v2)
result4 = reg4.fit()
print(result4.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     7.684
Date:                Wed, 22 Feb 2023   Prob (F-statistic):            0.00781
Time:                        14:41:05   Log-Likelihood:                -359.10
No. Observations:                  52   AIC:                             722.2
Df Residuals:                      50   BIC:                             726.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8411.1748    219.545     38.312      0.0

In [17]:
reg5 = smf.ols(formula = 'sales ~ price + summer_dummy', data = demand_v2)
result5 = reg5.fit()
print(result5.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     7.684
Date:                Wed, 22 Feb 2023   Prob (F-statistic):            0.00781
Time:                        14:41:05   Log-Likelihood:                -359.10
No. Observations:                  52   AIC:                             722.2
Df Residuals:                      50   BIC:                             726.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     2105.3159     29.848     70.534   

In [18]:
demand_v2.corr()

,vendor_id,week,summer_dummy,price,sales
vendor_id,NaN,NaN,NaN,NaN,NaN
week,NaN,1.000000,0.057703,0.057703,0.152865
summer_dummy,NaN,0.057703,1.000000,1.000000,0.364969
price,NaN,0.057703,1.000000,1.000000,0.364969
sales,NaN,0.152865,0.364969,0.364969,1.000000


#### We see that the summer_dummy and price are perfectly correlated. This is be because vendor 2 prices the ice cream based on the season, meaning he raises the price in summer and keeps the price low in winter.

### Question 3

#### If any of the vendors didn't systematically alter prices based on the season, we would not see bias in the price coefficient as we include or exclude the summer_dummy in the regression. Omitted variable bias applies when the variable that we have omitted is correlated with the sales. If the omitted variable (in this case summer_dummy) is not correlated to the sales at all, it would be irrelavant, and would not bias the coefficient at all. That said, adding summer_dummy may improve precision of the sales estimate by capturing some of the seasonal variation. Precision increases with decrease in std err.
